In [1]:
import pandas as pd
import random
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from pyvis.network import Network
import community.community_louvain as community_louvain
from tqdm import tqdm

---

## Importation des données

In [2]:
project = pd.read_csv("../Data/CSV/Projects/project.csv", sep=";")

In [3]:
organization = pd.read_csv("../Data/CSV/Projects/organization.csv", sep=";")

On importera les pubications après

In [4]:
print(project.shape)
print(organization.shape)

(35381, 20)
(177190, 25)


---

## Création du graph

On récupère les ID unique des projets

In [6]:
project_id = list(project["id"].unique())

Pour chaque projet je récupère les id des organisations qui y participent. Tout ça dans un dictionnaire.

In [7]:
projet_orga = {}
for projet in project_id:
    projet_orga[projet] = list(organization[organization["projectID"] == projet]["organisationID"])

Création d'un graph vide

In [8]:
G = nx.Graph()

Fonction de création du graph (prendre plus d'une heure à faire tourner)

In [9]:
########### /!\ TOURNE INDEFINIEMENT /!\ ##############

yes_counter = 0
for i, tq in zip(range(1, len(projet_orga.keys())), tqdm(range(len(projet_orga.keys())))):
#     print(projet_orga[project_id[i]])
    for t in range(len(projet_orga.keys())):
#         G.add_node(project_id[i - 1])
        G.add_node(project_id[i - 1], label=project_id[i - 1])
        for y in projet_orga[project_id[i - 1]]:
#         print(f"i: {i} | i+1: {projet_orga[project_id[i+1]]} | y: {y}")
            if G.has_edge(project_id[i - 1], project_id[t]) is False:
                for x in projet_orga[project_id[t]]:
            
                    if y == x:
                        yes_counter += 1
                        G.add_edge(project_id[i-1], project_id[t])

G.remove_edges_from(nx.selfloop_edges(G))
# On supprime les boucles du graph car cela n'apporte pas d'information et cela prend de la place

  0%|                                                                              | 8/35381 [00:02<3:31:10,  2.79it/s]


KeyboardInterrupt: 

Pour sauvegarder le graph au format .pickle

In [ ]:
pickle.dump(G, open('./graphs_save/graphe_3_labeled.pickle', 'wb'))

Pour importer un graph sauvegardé au format .pickle

In [10]:
 G = pickle.load(open("./graphs_save/graphe_3_labeled.pickle", "rb"))

In [11]:
G.number_of_nodes()

35381

In [12]:
G.number_of_edges()

9982209

---

## Détection de communauté

In [ ]:
partition = community_louvain.best_partition(G)

On ajoute les communauté en tant qu'attributs aux noeuds du graph

In [ ]:
nx.set_node_attributes(G, partition, 'partition')

Pour sauvegarder le graph en format gexf pour l'importer sur Gephi

In [ ]:
nx.write_gexf(G, "./graphs_save/graphe_3_labeled.gexf")

On crée un DataFrame avec en index les id des noeuds et leur communauté

In [ ]:
partition_df = pd.DataFrame.from_dict(partition, orient="index").astype({0:"int32"})

On ajoute les communautés au DataFrame de données

In [ ]:
project = project.join(partition_sorted, on="id")
project.rename({0:"Community"}, axis=1, inplace=True)

On calcule la proportion du chaque communauté par rapport à tous le data set 

In [ ]:
proportions = project['Community'].value_counts(normalize=True)